In [2]:
import pandas as pd
import numpy  as np

In [3]:
%%time
df = pd.read_excel('E:/Host/Data/Bees/Finance_ML/gazp15.xlsx', decimal='.', sep = ',', encoding='latin-1', dayfirst=True)



Wall time: 13.5 s


In [4]:
df

,Date,Time,Open,High,Low,Close,Volume
0,2006-01-23,10:30:00,239.000000,239.000000,220.009995,220.500000,681738
1,2006-01-23,10:45:00,220.330002,221.440002,219.330002,220.500000,698500
2,2006-01-23,11:00:00,220.500000,220.940002,219.710007,220.940002,141357
3,2006-01-23,11:15:00,220.899994,220.899994,219.800003,220.460007,154324
4,2006-01-23,11:30:00,220.460007,220.970001,220.009995,220.500000,64555
...,...,...,...,...,...,...,...
103191,2018-04-09,17:45:00,135.029999,135.399994,134.699997,135.100006,1097130
103192,2018-04-09,18:00:00,135.080002,135.660004,134.899994,135.009995,736800
103193,2018-04-09,18:15:00,135.000000,135.130005,134.850006,134.929993,731980
103194,2018-04-09,18:30:00,134.919998,134.979996,134.699997,134.850006,496610


In [14]:
#change type Time
df['Time'] =  pd.to_datetime(df['Time'], format='%H:%M:%S')
df = df.drop_duplicates(['Date', 'Time'], ignore_index = True)

#mark 12:45, 10:30
df['12:45'] = np.where(df.Time == '1900-01-01 12:45:00', 1, 0)
df['10:30'] = np.where(df.Time == '1900-01-01 10:30:00', 1, 0)
#df[df['12:45']==1]
df1 = df[(df['12:45']==1)]
df2 = df[(df['10:30']==1)]

#merg df1 df2 ---> df3
df3 = pd.merge(df1, df2, how = 'inner', on = 'Date', suffixes=('', '_x'), right_index = True)
df3 = df3.drop_duplicates('Date')
display(df3.shape)
#df3[df3['Date'] >= '2008-10-23']

#Создали return df3
#live 3 columns
df3['return'] = (df3['Close_x'].shift(-1)/df3['Close']-1)*100
col = ['Date',
       'Time', 
        'return'
 ]
df3 = df3[col]
df3 = df3.drop_duplicates('Date')
display(df3.shape)


(3035, 17)

(3035, 3)

In [22]:
%%time
# Создаем блоки утра namber = 16 
all_15 = pd.DataFrame()
for i in range(15, len(df)):
    if df.loc[i]['12:45'] == 1:
        df_15 = df.loc[i-15:i]
        all_15 = all_15.append(df_15)
display(all_15.shape)        
display(len(all_15)/16)        

(48576, 9)

3036.0

Wall time: 27.9 s


In [106]:
#create pl-mi
df3['return_pm'] = np.where(df3['return']>0, 1, 0)
df3

,Date,Time,return,return_pm
9,2006-01-23,1900-01-01 12:45:00,-0.212815,0
42,2006-01-24,1900-01-01 12:45:00,4.005081,1
75,2006-01-25,1900-01-01 12:45:00,0.705781,1
108,2006-01-26,1900-01-01 12:45:00,-0.916270,0
141,2006-01-27,1900-01-01 12:45:00,2.561247,1
...,...,...,...,...
102743,2018-04-03,1900-01-01 12:45:00,0.577086,1
102779,2018-04-04,1900-01-01 12:45:00,0.858551,1
102815,2018-04-05,1900-01-01 12:45:00,0.071117,1
102851,2018-04-06,1900-01-01 12:45:00,-4.466555,0


In [107]:
%%time
#create data+16close_morning for DATA
close = np.array(all_15['Close'])
close = np.reshape(close, (int(len(all_15)/16), 16))
data = np.array(all_15['Date'])
data = np.reshape(data, (int(len(all_15)/16), 16))
data = data[:,-1]
data_close = pd.concat([pd.DataFrame(data, columns=['Date']), pd.DataFrame(close)], axis=1)
data_close.shape

#merg data_close and df3 for DATA ИТОГОВАЯ
display(df3.shape)
display(data_close.shape)
df_merg = pd.merge(df3, data_close, how='inner', on='Date')
display(df_merg.shape)
df_merg.fillna(0, inplace = True)
display(df_merg.shape)
df_merg.drop_duplicates(subset = ['Date'], inplace = True)
display(df_merg.shape)
display('df_merg')
display(df_merg)


(3035, 4)

(3036, 17)

(3034, 20)

(3034, 20)

(3034, 20)

'df_merg'

,Date,Time,return,return_pm,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,2006-01-24,1900-01-01 12:45:00,4.005081,1,219.399994,219.610001,219.490005,220.000000,218.960007,218.889999,220.380005,220.490005,220.440002,220.500000,220.199997,219.800003,220.250000,220.449997,220.500000,220.470001
1,2006-01-25,1900-01-01 12:45:00,0.705781,1,222.449997,222.869995,222.899994,223.000000,223.889999,224.000000,229.300003,229.000000,229.550003,228.800003,227.880005,227.869995,226.110001,226.600006,227.210007,226.699997
2,2006-01-26,1900-01-01 12:45:00,-0.916270,0,228.500000,228.600006,228.449997,228.770004,228.699997,228.380005,228.300003,228.500000,227.800003,227.600006,227.789993,227.550003,227.880005,227.789993,228.000000,228.100006
3,2006-01-27,1900-01-01 12:45:00,2.561247,1,224.500000,225.500000,225.160004,225.389999,225.160004,224.470001,226.009995,225.300003,225.839996,225.710007,225.710007,225.600006,225.800003,225.399994,225.550003,224.500000
4,2006-01-30,1900-01-01 12:45:00,-2.971174,0,227.899994,227.399994,227.449997,227.500000,227.729996,228.750000,230.250000,228.839996,228.500000,228.750000,228.300003,227.460007,228.000000,227.279999,225.350006,225.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029,2018-04-03,1900-01-01 12:45:00,0.577086,1,140.600006,141.360001,140.910004,140.320007,140.860001,140.649994,140.240005,140.080002,140.240005,140.429993,140.470001,140.559998,140.490005,140.589996,140.539993,140.360001
3030,2018-04-04,1900-01-01 12:45:00,0.858551,1,140.699997,140.610001,140.660004,141.130005,140.669998,140.869995,141.169998,140.419998,140.229996,140.399994,140.259995,140.169998,139.520004,139.470001,139.529999,139.770004
3031,2018-04-05,1900-01-01 12:45:00,0.071117,1,139.740005,140.070007,140.190002,140.179993,141.199997,141.139999,140.970001,140.949997,140.910004,140.720001,140.580002,140.639999,140.720001,140.589996,140.839996,140.600006
3032,2018-04-06,1900-01-01 12:45:00,-4.466555,0,140.610001,140.309998,140.250000,140.500000,140.139999,140.509995,140.699997,140.679993,140.979996,140.839996,141.220001,141.070007,141.330002,141.740005,141.509995,141.720001


Wall time: 91 ms


In [ ]:
#normal on rows
# display(close100[0:3])
# scaler = MinMaxScaler(feature_range=(0,1))
# close100_n = scaler.fit_transform(close100.T)
# close100_n = close100_n.T
# display(close100_n)
# display(close100_n.shape)